## Yggdrasil Model Visualization

### Libraries Used
As a note, please be sure to have model_loader.py in the same folder as this

In [ ]:
import jupyterlab_nodeeditor as jlne
import ipywidgets
from yggdrasil.runner import YggRunner
from model_loader import dict_conversion, load_model

### Helper Function: Clean Bytes
Input: A 'bytes' type of data

Output: A cleaned 'string' conversion of the input

The reason behind this is just to make the output legible by removing quotations and any useless characters.

In [ ]:
# This will be a cleaner for the sidebar display
def clean_bytes(ibytes):
    sbytes = str(ibytes)[1:].strip("'")
    if "\\n" in sbytes:
        clean = sbytes[:-2]
    else:
        clean = sbytes
    return clean

### Setup
Initialize the Node Editor instance 'ne' and load a model into it.

Use the filepath *yggdrasil_test_models/model_trifecta.yml* for a sample model.

As a note, the Trifecta sample model consists of three models
1. Model RNG - A model which outputs a random number 4 times (1 per line in the input text file)
2. Model Reader - Reads said input text file line by line
3. Model ROTXX - Rotates each line by the number generated and outputs it encoding in a ROT-XX format (think of ROT13 encoding, where each letter in the alphabet is moved 13 places, but here the number is random)

**Make sure you manually right click and add all of your desired components once the editor appears.**

As of March 30th, 2022, there is a bug that does not allow for the components to be linked. We are working on fixing that ASAP but for production it will still execute normally.

In [ ]:
ne = jlne.NodeEditor()
load_model(ne)

### Callback Classes and Display
This is where things get a bit messy and will need to be adjusted based on your specific model.

The class name doesn't matter (*CBJLNE* - CallBack JLNE) so long as it is called properly. The comments in the code will explain each portion to better help the flow of this example.

*args[0]* is where the data in the runner is stored (i.e. the data that is being moved around, in the sample model's case, the number, line of text, and the rotated output)

*Note: You will see that we use the 3, 4, and 5 indices for the children but this modifies the sixth, seventh, and eighth rows in the tab. This is due to the Inputs/Outputs in JLNE spacing out their actual name to the next line. For example, under Inputs in Tab 3 it says "Slot 1: input_rotxx (temp_in0)" in the next row, but this is still all in the 2nd ipywidgets Label, just spread out for visual clarity in case of multiple Inputs/Outputs. If you need more to display, continue the index trend and do not worry about the rows. Be sure to add them to the initial length check as well so all rows are created properly.*

In [ ]:
# Create the callback class and have the third component be updated with live information
# You can add/remove/adjust functions that best suit what data you want to output
# As of March 30th, 2022 there is an issue with connected components in JLNE but the third tab will display the correct information regardless
class CBJLNE:
    # We check if there are 5 elements in the tab already. On initialization, this is not the case, so we add 3 more to the third tab,
    # which is index number 2, and add in filler labels until they are overwritten with the actual data
    if len(ne.node_editor.nodes[2].display_element.children) <= 5:
        for i in range(3):
            ne.node_editor.nodes[2].display_element.children += (ipywidgets.Label("Initializing"),)
    
    # The initialization just puts a Label in for this class
    def __init__(self):
        self.label = ipywidgets.Label()
    
    # Display Number pulls the number from Model RNG to display in the sixth row in the third tab
    def display_number(self, *args, **kwargs):
        ne.node_editor.nodes[2].display_element.children[3].value = f"Number: {args[0].args[0]}"
    
    # Display String pulls the line text from Model Reader to display in the seventh row in the third tab and cleans it using the previously defined Clean Bytes
    def display_string(self, *args, **kwargs):
        print(clean_bytes(args[0].args))
        ne.node_editor.nodes[2].display_element.children[4].value = f"String: {clean_bytes(args[0].args)}"
    
    # Display Rotated pulls the output from the Trifecta Model (i.e. the Rotated data) to show in the eighth row in the third tab
    def display_rotated(self, *args, **kwargs):
        ne.node_editor.nodes[2].display_element.children[5].value = f"Rotated: {args[0].args}"

# Make sure to call the class with a name that is easy to remember for use
display = CBJLNE()

### Connection Drivers
By having a runner on the model path, we can execute the YAML file directly inside the notebook.

In [ ]:
# Create a runner for the model, adjust the filepath for wherever the model is located
runner = YggRunner("yggdrasil_test_models/model_trifecta.yml")

You will need to uncomment the below line and run it to see what your YAML looks like as a runner in order to properly match the class functions to their connections

In [ ]:
# runner.connectiondrivers

Each connectiondriver dictionary key corresponds to a connection in your model.
For the sample model, we have
1. The text file connected to the Reader model
2. The random number and line of text connected to the ROTXX model
3. The output of the ROTXX model (i.e. the rotated string)

To utilize realtime updating with a Yggdrasil Runner, we need ot add in a 'callbacks' key to each connectiondriver. The value of this key will be the class function corresponding to the data you want to show casted to a list.

To break it down further, the value must be the class class with brackets around it so it is recognized by the Runner. This will allow the Runner to call back to that class function whenever the model is run. This is why you can create/remove/adjust any functions from the sample set to best fit your model. 

**Be sure to call the correct function on the correct connection**

In [ ]:
# Add in the callback functions to the connection drivers for execution
runner.connectiondrivers['model_reader:inputReader']['callbacks'] = [display.display_string]
runner.connectiondrivers['model_rng:outputRNG,model_reader:outputReader_to_model_rotxx:input_rotxx']['callbacks'] = [display.display_number]
runner.connectiondrivers['model_rotxx:output_rotxx']['callbacks'] = [display.display_rotated]

Finally we can execute the Runner to watch it all flow

In [ ]:
# Run the runner to see the data update in realtime
runner.run()